In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
import os
import numpy as np

/Users/yangyunjie1/opt/anaconda3/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
conn = sqlite3.connect('data.db')
cur = conn.cursor()
cur.execute("SELECT * FROM ohlc")
data = cur.fetchall()

In [3]:
allPrices = pd.DataFrame(
    data,
    columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD', 'token', 'chain'],
)
conn.close()

In [4]:
allPrices

,ts,open,high,low,close,volume,volumeUSD,token,chain
0,2021-11-01 00:00:00,61421.37,61669.14,61239.60,61343.68,256.433869,1.575751e+07,BTC,BTC
1,2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.481185,2.044558e+07,BTC,BTC
2,2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC
3,2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC
4,2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC
...,...,...,...,...,...,...,...,...,...
11622,2021-12-14 20:00:00,178.99,181.27,178.72,180.28,628.699000,1.130815e+05,COMP,ETH
11623,2021-12-14 21:00:00,180.15,183.76,179.96,183.43,2453.455000,4.447119e+05,COMP,ETH
11624,2021-12-14 22:00:00,183.31,186.74,182.60,185.24,2086.017000,3.865710e+05,COMP,ETH
11625,2021-12-14 23:00:00,185.22,186.09,183.88,185.08,416.028000,7.698720e+04,COMP,ETH


In [5]:
tokens = allPrices.token.unique()
tokens

array(['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP'], dtype=object)

In [6]:
i = -1
while True:
    if pd.to_datetime(allPrices['ts']).iloc[i].weekday() > 0:
        i=i-1
    else:
        firstday = i-24*7-23
        break

In [7]:
from_date = pd.to_datetime(allPrices['ts'].iloc[firstday]).date()
from_date

datetime.date(2021, 12, 6)

In [8]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [9]:
run_dates = [(last_dow(from_date, 9) + timedelta(days=7 * x)) for x in range(1)]
tokens = tokens
connection_string = 'sqlite:///data.db'

In [10]:
run_dates

[datetime.date(2021, 12, 8)]

In [11]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [12]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'HW4_template_Yunjie_Yang.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=14)}',
                'token': token,
                'connection_string': connection_string,
            }
        )

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

Executing:   0%|          | 0/23 [00:00<?, ?cell/s]

In [60]:
nbs = sb.read_notebooks('outputs')

In [101]:
summary = pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'return': np.nanmean(scrap['prices'].data['return']),
        'volatility': scrap['prices'].data.volatility.mean(),
        'maxMarkdown': scrap['prices'].data.maxMarkdown.max(),
        'close':scrap['prices'].data.close.max() }) 
    for book, scrap in nbs.notebook_scraps.items()
])

avg hourly return by run
avg volatility by run
maximum drawdown over the entire period of each run
range of last close price - first close price

In [102]:
summary

,token,from_date,to_date,return,volatility,maxMarkdown,close
0,AAVE,2021-12-08,2021-12-22,-0.000550,0.190581,8.7810,203.6060
1,ADA,2021-12-08,2021-12-22,-0.000483,0.192630,0.0614,1.4095
2,ATOM,2021-12-08,2021-12-22,-0.000601,0.236192,1.2700,25.5800
3,AVAX,2021-12-08,2021-12-22,-0.000113,0.233338,4.6700,93.8700
4,BTC,2021-12-08,2021-12-22,-0.000264,0.133991,2136.8300,50781.0500
5,COMP,2021-12-08,2021-12-22,-0.001065,0.227255,24.5600,226.4200
6,CRV,2021-12-08,2021-12-22,-0.001111,0.256654,0.2191,4.2038
7,DOT,2021-12-08,2021-12-22,-0.000884,0.214344,1.7400,30.5510
8,ETH,2021-12-08,2021-12-22,-0.000654,0.160270,211.6900,4440.5100
9,SOL,2021-12-08,2021-12-22,-0.001005,0.197032,10.7800,194.7920
